In [12]:
import vk
import pandas as pd
import time
from datetime import datetime
from tqdm import tqdm, tqdm_pandas

In [13]:
# opennig vk session with access token.
session = vk.Session(access_token='0dfed272d8c2c0882f26fabaee86c1c23eca901348c1e8476f93a1fd83fe06b4b9fc4070f7c0f1aa269f6')
api = vk.API(session)
# example use with id, my id
api.users.get(user_ids='architectofreality' )

[{'first_name': 'Alexander', 'last_name': 'Ovchinnikov', 'uid': 182927367}]

In [14]:
%%time
glaukoma_rus=api.newsfeed.search(q='glaukoma',count=200, end_time=1512378120, start_time=1512378121-86400)

Wall time: 333 ms


In [4]:
%%time
new_df = pd.read_csv(r'D:\vk_all_keywors_10_years_vk8_pretty.csv')

In [15]:
new_df.shape

(260155, 28)

In [16]:
# getting the comments of posts
def commentscrapper(x):
    uid = x['user_id']
    pid = x['post_id']
    
    try:
        time.sleep(2)
        likes = api.likes.getList(type='post', item_id = pid, owner_id = uid)
    # if we need to manually stop the program
    except (KeyboardInterrupt, SystemExit):
        raise 
    except VkAPIError as error:
        # check if error message is due to access deny for the post likes
        if error.message == 'Access denied: access restriction':
            likes = 'Access denied'
    # catch any other error, print the error
    except Exception as error:
        print('error occured on likes \n',error)
        likes = 'Error while scraping'
        raise
        
    try:
        time.sleep(2)
        reposts = api.likes.getList(type='post', item_id = pid, owner_id = uid, filter = 'copies')
    # if we need to manually stop the program
    except (KeyboardInterrupt, SystemExit):
        raise
    except VkAPIError as error:
        # check if error message is due to access deny for the post reposts
        if error.message == 'Access denied: access restriction':
            reposts = 'Access denied'
    # catch any other error, print the error
    except Exception as error:
        print('error occured on reposts \n',error)
        reposts = 'Error while scraping'
        raise
        
    # handling VKAPIERRORS for comments    
    try: 
        time.sleep(2) 
        coms = api.wall.getComments(owner_id = uid, post_id = pid, need_likes = 1)
    # if we need to manually stop the program
    except (KeyboardInterrupt, SystemExit):
        raise            
    except VkAPIError as error:
        # check if error message is due to access deny for the post comments
        if error.message == 'Access to post comments denied' or error.message == "Access denied: access restriction":
            pass
        # check if error is due to post was not found (probably was deleted while the algorithm ran)
        elif error.message == 'Access denied: post was not found check post_id param':
            coms = '1Post was not found'
        elif error.message == 'Access denied: post was deleted':
            coms = '1Post was deleted'            
        # print the error message in other cases
        else:
            print(error)
        coms = '1Access Denied'
    except Exception as error:
        print('error occured on comments \n',error)
        coments = '1Error while scraping'
        raise    
    return {'comments':coms[1:],'users_liked': likes,'users_reposted': reposts,'user_id': uid, 'post_id':pid}

In [17]:
# import all error types vfrom vk in case problem occurs with api connection
from vk.exceptions import VkAPIError,VkException,VkAuthError

In [ ]:
comments_likes_reposts = new_df.copy()#pd.DataFrame()
comments_likes_reposts['comments'] = None
comments_likes_reposts['users_liked'] = None
comments_likes_reposts['users_reposted'] = None
for i in tqdm(range(new_df.shape[0])):
    # in case a timeout error or something else occurs try on the same post till no error occurs 
    # avoid the same error if it occurs on the same post more than 10 times
    count = 0
    while count<=10:
        try:
            clr = commentscrapper(new_df.iloc[i])
            comments_likes_reposts.set_value(i,'comments' ,clr['comments'] )     
            comments_likes_reposts.set_value(i,'users_liked', clr['users_liked'])          
            comments_likes_reposts.set_value(i,'users_reposted', clr['users_reposted'])
            break
        # in case requests per second exceed limit  sleep for 10 mins
        except VkAPIError as error:
            if error.message == 'Too many requests per second':
                print('timeout error occured sleep for 1 min')
                time.sleep(60)
            else:
                print(error)
        # if we need to manually stop the program
        except (KeyboardInterrupt, SystemExit):
            raise
        # catch any other error print message sleep for 10 seconds re-enter WHILE loop
        except Exception as error:
            print('error occured during the main loop \n sleep 10 sec \n',error,'\n count of same error: ',count)
            count += 1
            time.sleep(10)
        if i%5000 == 0:
            comments_likes_reposts.to_csv(r'comments_likes_reposts.csv')



  0%|                                                                                       | 0/260155 [00:00<?, ?it/s]c:\users\mher.matevosyan\anaconda\envs\vk\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
c:\users\mher.matevosyan\anaconda\envs\vk\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
c:\users\mher.matevosyan\anaconda\envs\vk\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  

  0%|                                                                           | 1/260155 [00:06<454:23:34,  6.29s/it]
Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\users

In [ ]:
comments_likes_reposts.head()

In [ ]:
comments_likes_reposts.to_csv('vk_all_leyords_10_years_vk8_comments_likes_reposts.csv')

In [ ]:
short_df = comments_likes_reposts.copy()

In [ ]:
pd.options.display.max_columns = 100
short_df.head()

In [ ]:
# remove posts with more than 300 characters and posts with no characters
short_df['post']=short_df.text.apply(lambda x: x if len(x)<300 and len(x) !=0 else 'Too Long')

In [ ]:
not_long = short_df[short_df['post']!='Too Long'].reset_index().drop('index', axis = 1)

In [ ]:
import re
list_of_animals = 'кот|кошка|котёнок|котяра|котяры|котяр|котище|котик|собака|пес|собачка|кобель|псина|сука|птица|пташка|птенцы|птенец|щенок'
def find_animal(x):
    my_list = re.findall(list_of_animals,x['post'])
    if len(my_list) == 0:
        return(True)
    else:
        return(False)

In [ ]:
new_df['animal'] = new_df.apply(find_animal,axis = 1)

In [ ]:
(new_df.animal == True).value_counts()

In [ ]:
short_df.to_csv(r'D:\vk_all_keywords_10_years_vk8_short.csv')